## Dividend Growth Investment Data

Dividend growth investing has been around for ages and has always been a widely appreciated way of receiving passive income. As with any other investment, an elaborated decision needs to be made which stocks to pick. Also as with any other investment: the higher the risk, the higher the potential reward. This dataset should support the decision for a portfolio selection driven by data.

This notebook will download all relevant data from robinhood.com and dividend.com for dividend investors.    
The idea is to calculate a most realistic risk/reward score.    
It can be merged with any other data set based on the symbol.   

The approach is to download everything from robinhood.com, filter for only tradable stocks that pay dividend, then add their fundamental data, add ratings that robinhood.com offer (including the text for a rating itself), add popularity (basically how often this stock is in portfolios of robinhood users and add news that are displayed on robinhood.com. Lastly go through each stock symbol and look for a corresponding page on dividend.com. It turns out that only 7% of the robinhood dividend paying stocks can be found on dividend.com.

In [1]:
# Imports
import time #to measure times
import json #to handle the json respons
import pandas as pd #for data wrangling
import numpy as np #for certain dtypes
from urllib.request import Request, urlopen #to perform URL get requests
from urllib.parse import urlencode #to URL encode a payload
from datetime import datetime, timedelta #for datetime conversions
from lxml import etree #to read the xml tree

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
#Functions

# This chunker function will help us loop through long lists of stock symbols without downloading every single one of them
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# This recursive function allows us to download all pages from a paginated robinhood API response until there is no next page left
def robinhood_download_all(url, i = 1, top=None):
    url = url.replace('com/ratings','com/midlands/ratings') #needed because the 'next' URL in an API response is actually wrong sometimes
    data = json.load(urlopen(Request(url)))
    df = pd.DataFrame.from_records(data['results'])
    print('\r{}. page downloaded: {}'.format(i,url),end='')
    n = data['next']
    if top:
        if i >= top: 
            return df
    if n:
        return pd.concat([df, robinhood_download_all(n, i+1, top)])
    else:
        return df

# This function loops through groups of a long list. We need this function for all API calls that allow a comma separated
# list as payload, but to attach the whole list would be too long. So we chunk it in batches.
def robinhood_download_list(base_url, l, length = 100, i = 1):
    result_dfs = []
    for group in chunker(l, length):
        print('\r{}. group downloaded: {}'.format(i,base_url),end='')
        i+=1
        result_dfs.append(
            pd.DataFrame.from_records(
                list(
                    filter(
                        None, 
                        json.load(
                            urlopen(
                                Request(base_url+','.join(group))
                            )
                        )['results']
                    )
                )
            )
        )
    return pd.concat(result_dfs)

# This functions converts columns that contain a $ sign
def convert (x):
    try:
        return float(x.strip('$'))
    except:
        return np.nan

In [4]:
# Download list of instruments
start_time = time.time()
instruments = robinhood_download_all('https://api.robinhood.com/instruments/')
print('\nTime elapsed for {}: {}'.format('downloading instruments',str(timedelta(seconds = time.time() - start_time))))

111. page downloaded: https://api.robinhood.com/instruments/?cursor=cD00NQ%3D%3D
Time elapsed for downloading instruments: 0:02:08.690918


In [5]:
# Keep only tradables
instruments = instruments[instruments['tradeable']==True]

In [6]:
# We'll need this number later
len_all_tradable_stocks = len(instruments)

In [7]:
print('Shape of instruments:',instruments.shape)

Shape of instruments: (7187, 20)


In [8]:
# Download fundamentals for instruments
start_time = time.time()
fundamentals = robinhood_download_list('https://api.robinhood.com/fundamentals/?symbols=', list(instruments['symbol']), length = 100)
print('\nTime elapsed for {}: {}'.format('downloading fundamentals',str(timedelta(seconds = time.time() - start_time))))

72. group downloaded: https://api.robinhood.com/fundamentals/?symbols=
Time elapsed for downloading fundamentals: 0:01:02.008592


In [9]:
print('Shape of fundamendals:',fundamentals.shape)

Shape of fundamendals: (7163, 20)


In [10]:
#Merge instruments and fundamentals
instruments = pd.merge(instruments, fundamentals, how='left', left_on='url', right_on='instrument')

In [11]:
# select only the instruments that pay a dividend (we'll handle the rest of the numeric columns later, but here it's needed already)
instruments['dividend_yield'] = pd.to_numeric(instruments['dividend_yield'])
instruments = instruments[instruments['dividend_yield'] > 0]

In [12]:
print("Shape of instruments paying dividend and their fundamentals: {}. That's {:.2f}% of all tradable stocks".format(instruments.shape, len(instruments) * 100 / len_all_tradable_stocks))

Shape of instruments paying dividend and their fundamentals: (3651, 40). That's 50.80% of all tradable stocks


In [13]:
# Download and flatten ratings
start_time = time.time()
ratings = robinhood_download_all('https://api.robinhood.com/midlands/ratings/')
ratings = pd.concat([ratings.summary.apply(pd.Series), ratings.drop('summary', axis=1)], axis=1)
print('\nTime elapsed for {}: {}'.format('downloading and flatten ratings',str(timedelta(seconds = time.time() - start_time))))

294. page downloaded: https://api.robinhood.com/midlands/ratings/?page=294
Time elapsed for downloading and flatten ratings: 0:03:44.648327


In [14]:
# Merge instruments and ratings
instruments = pd.merge(instruments, ratings, how='left', left_on='id', right_on='instrument_id')

In [15]:
# Download popularities
start_time = time.time()
l = list(instruments['id'])

popularities_dfs = []
i = 1
for group in chunker(l, 50):
    url = 'https://api.robinhood.com/instruments/popularity/?' + urlencode([('ids',','.join(group))])
    popularities_dfs.append(
        pd.DataFrame.from_records(
            list(
                filter(
                    None, 
                    json.load(
                        urlopen(
                            Request(url)
                        )
                    )['results']
                )
            )
        )
    )
    print('\r{}. popularity group downloaded: {}'.format(i,url),end='')
    i+=1
    time.sleep(0.01)
    # This wait was necessary in my case to not run into rate limits
    if (len(popularities_dfs)%50==0):
        print('\rwaiting...',end='')
        time.sleep(60)
popularities = pd.concat(popularities_dfs).reset_index(drop=True)
print('\nTime elapsed for {}: {}'.format('downloading popularities',str(timedelta(seconds = time.time() - start_time))))

74. popularity group downloaded: https://api.robinhood.com/instruments/popularity/?ids=08c418ea-41a1-4ca4-8127-2fb924999b09
Time elapsed for downloading popularities: 0:01:54.307169


In [16]:
# Merge instruments and popularities
instruments = pd.merge(instruments, popularities, how='left', left_on='instrument', right_on='instrument')

In [17]:
# Download news
start_time = time.time()
news = []
for i,row in instruments.iterrows():
    url = 'https://api.robinhood.com/midlands/news/'+row['symbol']+'/'
    print('\r{:}/{} news downloaded ({:.2f}%): {}'.format(i+1,len(instruments),(i+1)*100/len(instruments),url),end='')
    news.append(json.load(urlopen(Request(url)))['results'])
print('\nTime elapsed for {}: {}'.format('downloading news',str(timedelta(seconds = time.time() - start_time))))

3651/3651 news downloaded (100.00%): https://api.robinhood.com/midlands/news/UGE/
Time elapsed for downloading news: 0:36:57.108773


In [18]:
instruments['news'] = news

In [19]:
# load data from dividend.com
start_time = time.time()
n = datetime.now()
dividend_com_rows = [] # single rows will be added to one data frame later
payout_histories = {} # a sub dict for the entire history of payouts for each instrument
url = 'http://www.dividend.com/search/?q='
# The following line is needed because the source code of the dividend.com website doesn't close
# this comment properly. In order for the etree to properly read the html, we need to replace it
faulty_comment = '<!--[if lte IE 9]>\n<a class="mm-header-logo" href="http://www.dividend.com/"><img alt="Dividend logo" src="/assets/dividend-logo-358fc66b2fe52772c6dad898b8c40050.png" /></a>\n<!--<![endif]--><!-->\n'
# Counter for the not found ones
not_found=0
for i,row in instruments.iterrows():
    security = row['symbol']
    try:
        html = urlopen(Request(url+security)).read()
        html = html.decode().replace(faulty_comment,'')
        tree = etree.HTML(html)
        values = {}
        values['symbol'] = security
        
        # Get each piece of information from the correspoinding div on the website
        metrics = {
            'div_yield' : '//*[@id="stock-dashboard-collapse"]/div[2]/div[1]/div/div[1]',
            'annual_payout' : '//*[@id="stock-dashboard-collapse"]/div[2]/div[2]/div/div[1]',
            'payout_ratio' : '//*[@id="stock-dashboard-collapse"]/div[2]/div[3]/div/div[1]',
            'div_growth' : '//*[@id="stock-dashboard-collapse"]/div[2]/div[4]/div/div[1]',
            'price' : '//*[@id="snapshot-collapse"]/div[1]/div[1]/div/div',
            'annual_payout' : '//*[@id="stock-dashboard-collapse"]/div[2]/div[2]/div/div[1]',
            'percent_off_52_week_high' : '//*[@id="snapshot-collapse"]/div[2]/div[4]/div/div/span',
            'name' : '//*[@id="stock"]/div[4]/div/div/h1/span',
            'exchange' : '//*[@id="profile-collapse"]/div[1]/div[1]/span',
            'sector' : '//*[@id="profile-collapse"]/div[1]/div[2]/a',
            'industry' : '//*[@id="profile-collapse"]/div[1]/div[3]/a'
        }
        for metric, xpath in metrics.items():
            try:
                e = tree.xpath(xpath)[0]
            except:
                value = None
            try:
                value = float(e.text.format().strip('%$yrs'))
            except ValueError as ve:
                value = e.text 
            except:
                value = None
            metrics[metric] = {
                'xpath' : xpath,
                'value' : value
            }
            values[metric] = value
        value = None
        e = None
            
        #Load the payout history into a DataFrame
        try:
            table = tree.xpath('//*[@id="payout-history-table-collapse"]/div/table')[0]
            payout_history = pd.read_html(etree.tostring(table).decode())[0]
            date_fields = ['Declared Date','Ex-Dividend Date','Record Date','Pay Date ▼']

            for date_field in date_fields:
                payout_history[date_field] = pd.to_datetime(payout_history[date_field])
            payout_history['Payout Amount'] = payout_history['Payout Amount'].apply(convert)
            
            # Calculate the closest paymen as date and in days difference
            payout_history['recent_payment_days'] = payout_history['Pay Date ▼'].apply(lambda x: x.date() - n.date())
            days_series = payout_history['recent_payment_days']
            
            # we need the following if statement in case there are multiple future pay dates already planned
            if len(days_series[days_series>timedelta(days=0)]) > 0:
                recent_pay_date = payout_history['Pay Date ▼'][max(days_series[days_series>timedelta(days=0)].index)]
                recent_payment = payout_history['recent_payment_days'][max(days_series[days_series>timedelta(days=0)].index)]
            else:
                recent_pay_date = max(payout_history['Pay Date ▼'])
                recent_payment = max(payout_history['recent_payment_days'])
            
            payout_histories[security] = payout_history

            values['recent_pay_date'] = recent_pay_date
            values['recent_payment'] = recent_payment
            values['payout_history'] = payout_history
        except Exception as e:
            not_found+=1
            
            
        dividend_com_rows.append(values)
    except Exception as e:
        #print('\r'+security,'not found')
        not_found+=1
    print('\r{:>7} downloaded, {}/{} ({:.2f}%) instruments downloaded; {} ({:.2f}%) not found'.format(security,i+1,len(instruments),(i+1)*100/len(instruments),not_found, not_found*100/(i+1)),end='')
dividend_com = pd.DataFrame(dividend_com_rows)
print('\nTime elapsed for {}: {}'.format('downloading data from dividend.com',str(timedelta(seconds = time.time() - start_time))))

    UGE downloaded, 3651/3651 (100.00%) instruments downloaded; 3475 (95.18%) not found
Time elapsed for downloading data from dividend.com: 1:02:18.032782


In [20]:
# We have this information already in the instruments
dividend_com.drop(['name','sector'],axis=1,inplace=True)

In [21]:
# Merge instruments with dividend.com data
instruments = pd.merge(instruments, dividend_com, how='left', left_on='symbol', right_on='symbol')

#### Data Preparation

In [22]:
# a few useful derives
instruments['percent_buy_ratings'] = instruments.apply(lambda x:x['num_buy_ratings']/sum(x[['num_buy_ratings','num_hold_ratings','num_sell_ratings']]), axis = 1)
instruments['percent_sell_ratings'] = instruments.apply(lambda x:x['num_sell_ratings']/sum(x[['num_buy_ratings','num_hold_ratings','num_sell_ratings']]), axis = 1)

In [23]:
# defining order and type of columns
object_cols = [
    'symbol',
    'name',
    'simple_name',
    'ceo',
    'description',
    'id',
    'instrument',
    'instrument_id',
    'bloomberg_unique',
    'fundamentals',
    'splits',
    'quote',
    'url'
]

category_cols = [
    'country',
    'state',
    'headquarters_state',
    'headquarters_city',
    'market',
    'exchange',
    'sector',
    'industry',
    'type',
    'tradability',
    'tradeable'
]

float_cols = [
    'price',
    'open',
    'high',
    'low',
    'high_52_weeks',
    'low_52_weeks',
    'percent_off_52_week_high',
    'volume',
    'average_volume',
    'average_volume_2_weeks',
    'dividend_yield',
    'div_yield',
    'div_growth',
    'annual_payout',
    'payout_ratio',
    'num_employees',
    'year_founded',
    'shares_outstanding',
    'market_cap',
    'num_buy_ratings',
    'num_hold_ratings',
    'num_sell_ratings',
    'percent_buy_ratings',
    'percent_sell_ratings',
    'num_open_positions',
    'day_trade_ratio',
    'maintenance_ratio',
    'margin_initial_ratio',
    'pe_ratio',
    'min_tick_size'
]

date_time_cols = [
    'list_date',
    'recent_pay_date'
]

#### Applying the defined types

In [24]:
instruments[float_cols] = instruments[float_cols].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [25]:
for col in category_cols:
    instruments[col] = instruments[col].astype('category')

In [26]:
instruments[date_time_cols] = instruments[date_time_cols].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [27]:
# ordering columns
instruments = instruments[object_cols + category_cols + float_cols + date_time_cols + list(set(list(instruments.columns ))- set(object_cols + category_cols + float_cols + date_time_cols))]

In [28]:
print(instruments.dtypes)

symbol                               object
name                                 object
simple_name                          object
ceo                                  object
description                          object
id                                   object
instrument                           object
instrument_id                        object
bloomberg_unique                     object
fundamentals                         object
splits                               object
quote                                object
url                                  object
country                            category
state                              category
headquarters_state                 category
headquarters_city                  category
market                             category
exchange                           category
sector                             category
industry                           category
type                               category
tradability                     

In [29]:
# save as pickle
instruments.to_pickle('instruments.p')

In [30]:
# save as json
instruments.to_json('instruments.json')